# SEMI-SUPERVISED CLASSIFICATION WITH GRAPH CONVOLUTIONAL NETWORKS


**Name** - B Pravena

**SRN** - PES2UG19CS076

**Section** - B

Dataset used is **CORA**.

The Cora dataset consists of 2708 scientific publications classified into one of seven classes. 

The citation network consists of 5429 links. 

Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Installing and importing Dependencies

In [2]:
!pip install tensorflow==2.3 spektral==0.6.1

In [3]:
import numpy as np
import os
import networkx as nx
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

import spektral
from tensorflow import keras 
from tensorflow.python.ops import gen_sparse_ops
from spektral.layers import GraphConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

# Data Preprocessing

In **cora.content** file -:

The first element indicates the node name. 
The second until the last second elements indicate the node features.
The last element indicates the label of that particular node.

In **cora.cites** file -:

Each line indicates the tuple of connected nodes

In [4]:
#loading the data

all_data = []
all_edges = []

for root,dirs,files in os.walk('/content/gdrive/MyDrive/cora'):
    for file in files:
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

                
#Data is shuffled because the raw data is ordered based on the label
random_state = 77
all_data = shuffle(all_data,random_state=random_state)

# Parsing the Data

In [5]:
labels = []
nodes = []
X = []

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(elements[-1])
    X.append(elements[1:-1])
    nodes.append(elements[0])

X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {'Reinforcement_Learning', 'Case_Based', 'Neural_Networks', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Rule_Learning', 'Theory'}

Number of classes:  7


#Data splitting

In [6]:
def limit_data(labels,limit=20,val_num=500,test_num=1000):
    label_counter = dict((l, 0) for l in labels)
    train_idx = []

    for i in range(len(labels)):
        label = labels[i]
        if label_counter[label]<limit:
            #add the example to the training data
            train_idx.append(i)
            label_counter[label]+=1
        
        #exit the loop once we found 20 examples for each class
        if all(count == limit for count in label_counter.values()):
            break
    
    #get the indices that do not go to traning data
    rest_idx = [x for x in range(len(labels)) if x not in train_idx]
    #get the first val_num
    val_idx = rest_idx[:val_num]
    test_idx = rest_idx[val_num:(val_num+test_num)]
    return train_idx, val_idx,test_idx


train_idx,val_idx,test_idx = limit_data(labels)

In [7]:
#set the mask
train_mask = np.zeros((N,),dtype=bool)
train_mask[train_idx] = True

val_mask = np.zeros((N,),dtype=bool)
val_mask[val_idx] = True

test_mask = np.zeros((N,),dtype=bool)
test_mask[test_idx] = True

# One hot Encoding

In [8]:
def encode_label(labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return labels, label_encoder.classes_

labels_encoded, classes = encode_label(labels)

# Building Graph on NetworkX

In [9]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)

#obtain the adjacency matrix (A)
A = nx.adjacency_matrix(G)
print('Graph info: ', nx.info(G))

Graph info:  Graph with 2708 nodes and 5278 edges


# Building and Traing GCN

In [10]:
# Parameters
channels = 16           # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 100           # Number of training epochs
es_patience = 10        # Patience for early stopping

# Preprocessing operations
A = GraphConv.preprocess(A).astype('f4')

# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GraphConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GraphConv(num_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])

In [11]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 1433)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_conv (GraphConv)          (None, 16)           22928       dropout[0][0]                    
                                                                 input_2[0][0]         

In [12]:
# Train model
validation_data = ([X, A], labels_encoded, val_mask)
model.fit([X, A],
          labels_encoded,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True),
          ])

Epoch 1/100
1/1 [==============================] - 0s 482ms/step - loss: 0.1166 - acc: 0.1286 - val_loss: 0.3683 - val_acc: 0.1920
Epoch 2/100
1/1 [==============================] - 0s 82ms/step - loss: 0.1099 - acc: 0.3071 - val_loss: 0.3586 - val_acc: 0.3300
Epoch 3/100
1/1 [==============================] - 0s 75ms/step - loss: 0.1034 - acc: 0.4500 - val_loss: 0.3496 - val_acc: 0.3900
Epoch 4/100
1/1 [==============================] - 0s 80ms/step - loss: 0.0985 - acc: 0.5786 - val_loss: 0.3413 - val_acc: 0.3760
Epoch 5/100
1/1 [==============================] - 0s 75ms/step - loss: 0.0931 - acc: 0.5714 - val_loss: 0.3341 - val_acc: 0.3800
Epoch 6/100
1/1 [==============================] - 0s 126ms/step - loss: 0.0887 - acc: 0.6357 - val_loss: 0.3275 - val_acc: 0.3740
Epoch 7/100
1/1 [==============================] - 0s 116ms/step - loss: 0.0840 - acc: 0.6571 - val_loss: 0.3221 - val_acc: 0.3880
Epoch 8/100
1/1 [==============================] - 0s 119ms/step - loss: 0.0807 - acc: 

# Model Evaluation

In [13]:
X_te = X[test_mask]
A_te = A[test_mask,:][:,test_mask]
y_te = labels_encoded[test_mask]

y_pred = model.predict([X_te, A_te], batch_size=N)
report = classification_report(np.argmax(y_te,axis=1), np.argmax(y_pred,axis=1), target_names=classes)
print('GCN Classification Report: \n {}'.format(report))

GCN Classification Report: 
                         precision    recall  f1-score   support

            Case_Based       0.67      0.78      0.72       114
    Genetic_Algorithms       0.86      0.86      0.86       156
       Neural_Networks       0.81      0.70      0.75       290
 Probabilistic_Methods       0.80      0.73      0.77       172
Reinforcement_Learning       0.70      0.79      0.74        85
         Rule_Learning       0.58      0.78      0.67        60
                Theory       0.60      0.62      0.61       123

              accuracy                           0.74      1000
             macro avg       0.72      0.75      0.73      1000
          weighted avg       0.75      0.74      0.74      1000

